# Quick start

Strategy similar to strategies on the legacy version of Quentiacs

* [User account](https://quantiacs.io/personalpage/homepage)
* [Documentation](https://quantiacs.io/documentation/en/)


> **Strategy idea.** We will open positions in futures where two moving averages of the closing prices intersect


> The trading algorithm uses financial data to form the weights, in proportion to which the capital is distributed. A positive weight means a long position (buy), a negative value means a short position (sell).

> For each date, the algorithm calculates what portfolio weights should be at the opening of the next day's trading.

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) { return false; }
// disable widget scrolling

<IPython.core.display.Javascript object>

In [2]:
import xarray as xr

import qnt.ta as qnta
import qnt.backtester as qnbt
import qnt.data as qndata


def load_data(period):
    return qndata.futures_load_data(tail=period)


def strategy(data):
    close = data.sel(field='close')
    sma200 = qnta.sma(close, 200).isel(time=-1)
    sma20 = qnta.sma(close, 20).isel(time=-1)
    return xr.where(sma200 < sma20, 1, 0)


weights = qnbt.backtest(
    competition_type="futures",
    load_data=load_data,
    lookback_period=365,
    test_period=15 * 365,
    strategy=strategy
)

Run last pass...
Ok.
---
Run first pass...
Ok.
---
Load full data...
---
Run iterations...



100% (3873 of 3873) |####################| Elapsed Time: 0:00:32 Time:  0:00:32


Merge outputs...
Load data for cleanup and analysis...
Check missed dates...
Ok.
Normalization...
Done.
Write output: /root/fractions.nc.gz
---
Analyze results...
Check...
Check missed dates...
Ok.
Check sharpe ratio.


ERROR! The sharpe ratio is too low. -0.07815169818121354 < 1


Check correlation.

Ok. This strategy does not correlate with other strategies.
---
Calc global stats...
---
Calc stats per asset...
Build plots...
---
Select the asset (or leave blank to display the overall stats):


interactive(children=(Combobox(value='', description='asset', options=('', 'F_AD', 'F_AE', 'F_AH', 'F_AX', 'F_…

## Implementing an identical strategy that works quickly


```python
import xarray as xr

import qnt.ta as qnta
import qnt.data as qndata
import qnt.output as qnout

data = qndata.futures_load_data(tail=15 * 365)

close = data.sel(field='close')
sma200 = qnta.sma(close, 200)
sma20 = qnta.sma(close, 20)
weights = xr.where(sma200 < sma20, 1, 0)
weights = qnout.clean(weights, data)

qnout.check(weights, data)

qnout.write(weights)
```

## if it is necessary to use several data sets in the strategy

```python
import xarray as xr
import numpy as np

import qnt.ta as qnta
import qnt.backtester as qnbt
import qnt.data as qndata


def load_data(period):
    futures = qndata.futures_load_data(tail=period)
    crypto = qndata.crypto_load_data(tail=period)
    return {"futures": futures, "crypto": crypto}, futures.time.values


def window(data, max_date: np.datetime64, lookback_period: int):
    min_date = max_date - np.timedelta64(lookback_period, 'D')
    return {
        "futures": data['futures'].sel(time=slice(min_date, max_date)),
        "crypto": data['crypto'].sel(time=slice(min_date, max_date)),
    }


def strategy(data):
    close = data['futures'].sel(field='close')
    close_last = data['futures'].sel(field='close').shift(time=1)
    close_change = close - close_last

    close_crypto = data['crypto'].sel(field='close')
    close_last_crypto = data['crypto'].sel(field='close').shift(time=1)
    close_change_crypto = close_crypto - close_last_crypto

    # .isel(time=-1) get the last day
    sma200 = qnta.sma(close_change, 20).isel(time=-1)
    sma200_crypto = qnta.sma(close_change_crypto, 20).isel(time=-1).sel(asset='BTC')
    return xr.where(sma200 < sma200_crypto, 1, 0)


qnbt.backtest(
    competition_type="futures",
    load_data=load_data,
    lookback_period=365,
    test_period=7 * 365,
    strategy=strategy,
    window=window
)
```

# Guide with the most frequently required functions

## What libraries are there?

```python
# Import basic libraries.
import xarray as xr
import numpy as np
import pandas as pd

# Import quantnet libraries.
import qnt.data    as qndata
import qnt.output as output
import qnt.backtester as qnbt
import qnt.stepper as qnstepper
import qnt.stats   as qnstats
import qnt.graph   as qngraph
import qnt.ta      as qnta # indicators library
import qnt.forward_looking as qnfl # forward looking checking
import qnt.exposure as qne
```

## How to load data?

```python
data = qndata.futures.load_data(tail = 15*365, dims = ("time", "field", "asset"))
```

```python
data = qndata.cryptofutures.load_data(tail = 15*365, dims = ("time", "field", "asset"))
```

```python
data = qndata.crypto.load_data(tail = 15*365, dims = ("time", "field", "asset"))
```

```python
data = qndata.stocks.load_data(tail = 15*365, dims = ("time", "field", "asset"))
```

## How to view a list of all tickers?

```python
data.asset.to_pandas().to_list()
```

## What market data is there?

```python
data.field.to_pandas().to_list()
```

## How to unload a specific ticker?

```python
data = qndata.stocks.load_data(tail=15 * 365, assets=['NASDAQ:AAL', 'NASDAQ:AAPL', 'NASDAQ:ADBE'])
```

## How to use specific tickers?

```python
def get_data_filter(data, assets):
    filler = data.sel(asset=assets)
    return filler

get_data_filter(data, ['NASDAQ:AAL', 'NASDAQ:AAPL', 'NASDAQ:ADBE'])
```

## How to get the prices for the previous day?

```python
qnta.shift(data.sel(field="open"), periods=1)
```

or

```python
data.sel(field="open").shift(time=1)
```

## How do I get a list of the top 50 Sharpe stocks?

```python
def get_best_instruments(data, weights, top_size):
    top_period = 3 * 365
    stats_per_asset = qnstats.calc_stat(data, weights, per_asset=True)
    # calculate ranks of assets by "sharpe_ratio"
    ranks = (-stats_per_asset.sel(field='sharpe_ratio')).rank('asset')
    # Select top assets by rank which assets have 'top_period' days ago.
    # We assume, that the "sharpe_ratio" of these assets will be good in the next 'top_period' days
    rank = ranks.isel(time=-top_period)
    top = rank.where(rank <= top_size).dropna('asset').asset

    # select top stats
    top_stats = stats_per_asset.sel(asset=top.values)

    # print results
    print("SR tail of the top assets:")
    display(top_stats.sel(field='sharpe_ratio').to_pandas().tail())

    print("avg SR = ", top_stats[-top_period:].sel(field='sharpe_ratio').mean('asset')[-1].item())
    display(top_stats)
    return top_stats.coords['asset'].values

get_best_instruments(data, weights, 50)
```

## How can I check the results for only the top 50 Sharpe stocks?

Download the Top 50 Companies List and apply your strategy to it

```python
best_stocks = get_best_instruments(data, weights, 50)

qndata.stocks.load_data(tail=15 * 365, assets=best_stocks)
```

## How can prices be processed?

Example logarithm

```python
import numpy as np

high = np.log(data.sel(field="high"))
```

## How can you reduce the commission when trading?

```python
def get_lower_commissions(weights, rolling_time=6):
    return weights.rolling({"time": rolling_time}).max()

get_lower_commissions(weights, rolling_time=6)
```

## How to use technical analysis indicators?

for available indicators see the source code of the library (/qnt/ta)

## ATR

```python
def get_atr(data, days=14):
    high = data.sel(field='high') * 1.0 
    low = data.sel(field='low') * 1.0 
    close = data.sel(field='close') * 1.0

    return qnta.atr(high, low, close, days)

get_atr(data, days=14)
```

## EMA

```python
prices = data.sel(field="high")
prices_ema = qnta.ema(prices, 15)
```

## TRIX

```python
prices = data.sel(field="high")
prices_trix = qnta.trix(prices, 15)
```

## ADL and EMA

```python
adl = qnta.ad_line(data.sel(field="close")) * 1.0 
adl_ema = qnta.ema(adl, 18)
```

## How do you know if your strategy is good?

```python
qnout.check(weights, data)
```

or

```python
stat = qnstats.calc_stat(data, weights)
display(stat.to_pandas().tail())
```

or

```python
import qnt.graph   as qngraph
statistics = qnstats.calc_stat(data, weights)
display(statistics.to_pandas().tail())

performance = statistics.to_pandas()["equity"]
qngraph.make_plot_filled(performance.index, performance, name="PnL (Equity)", type="log")

display(statistics[-1:].sel(field = ["sharpe_ratio"]).transpose().to_pandas())
qnstats.print_correlation(weights, data)
qnstats.check_exposure(weights)

```

or

```python
import qnt.stats   as qnstats

def estimate_sharpe(data, weights_final):
    stat = qnstats.calc_stat(data, weights_final, slippage_factor=0.05)
    days = len(stat.coords["time"])
    returns = stat.loc[:, "relative_return"]
    
    sharpe_ratio = qnstats.calc_sharpe_ratio_annualized(
        returns,
        max_periods=days,
        min_periods=days).to_pandas().values[-1]
    
    print(f'Sharpe ratio = {sharpe_ratio}')

estimate_sharpe(data, weights)
```

## An example implementation using pandas series

```python
def get_price_pct_change(prices):
    prices_pandas = prices.to_pandas()
    assets = data.coords['asset'].values
    for asset in assets:
        prices_pandas[asset] = prices_pandas[asset].pct_change()
    return prices_pandas


prices = data.sel(field='close') * 1.0
prices_pct_change = xr.DataArray(get_price_pct_change(prices), prices.coords)

```

## How to submit a strategy to the competition?

Check that there is a check in the code

```python
qnout.check(weights, data)
```

if everything is ok, write the weights to the file

```python
output.write(weights)
```

In your **personal account**:

* **choose** a strategy;
* click on the "**Submit**" button;
* select the type of competition.

Further, the strategy will go to the сheck (Competition > Checking)
